In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

train_data = pd.read_csv(
    './DataSet/train.csv')

In [27]:
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [28]:
X = train_data.drop(['SalePrice'], axis=1) # feature
y = train_data[['SalePrice']] # target

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                              train_size=0.8, test_size=0.2,
                                                              random_state=42)

In [30]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# วิธีการหาคอลัมน์ Object ใช้หลักการง่ายในการวัดว่าColumnไหนเป็นเลข หรือ หมวดหมู่ โดยใช้การนับความจำนวนค่าที่ซ้ำกันในแต่ละคอลัมน์หากมีค่าซ้ำกันในคอลัมน์น้อยกว่า10 คือเป็นคอลัมน์หมวดหมู่ 
categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].nunique() < 10 and  #ตรวจสอบว่าคอลัมน์มีจำนวนค่าที่ไม่ซ้ำกันน้อยกว่า 10 ค่า ซึ่งมักบ่งบอกถึงคอลัมน์ประเภท Categorical
                    X_train[cname].dtype == "object"] #กำหนดให้คอลัมน์นั้นเป็น Object

# Select numerical columns
# วิธีการหาคอลัมน์ ตัวเลข โดยใช้หลักการเช็คแต่ละคอลัมน์ว่ามี type เป็น int หรือ float ไหมถ้าใช่คือเป็นคอลัมน์ตัวเลข
numerical_cols = [cname for cname in X_train.columns if
                  X_train[cname].dtype in ['int64', 'float64']]

In [31]:
X_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
254,255,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1066,1067,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal
638,639,30,RL,67.0,8777,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal
799,800,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal
380,381,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal


In [32]:
# Preprocessing for numerical data
# เตรียมข้อมูลตัวเลข
numerical_transformer = SimpleImputer(strategy='constant') #ใช้การเติมค่าที่ขาดหายไป โดยใช้วิธีการConstant(ค่าคงที่) ค่าเริ่มต้นมักเป็น 0

# Preprocessing for categorical data
# เตรียมข้อมูลหมวดหมู่
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  #ใช้วิธีการ most_frequent หมายถึงจะแทนค่าที่ขาดหายไปด้วยค่าที่พบบ่อยที่สุด
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # แปลงข้อมูลหมวดหมู่ให้เป็นตัวแปรเลขฐานสอง (Binary Variables) หลายตัว
])

In [33]:
# Bundle preprocessing for numerical and categorical data
from sklearn.metrics import r2_score
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
# กำหนด Model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                      ])

# Preprocessing of training data, fit model
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_test)

print('MAE:', mean_absolute_error(y_test, preds))


r2 = r2_score(y_test, preds)
print('R^2:', r2)

c:\Users\Sonto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py:382: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


MAE: 17916.41339041096
R^2: 0.8899644727216792


In [34]:
# Find The Accuracy percent
output = pd.DataFrame({'SalePrice': y_test['SalePrice'],
                       'Predicted_Price': preds,
                       'Error': y_test['SalePrice'] - preds})

In [36]:
# Calculate 'Accuracy'
Accuracy = len(output[output['Error'] <= 10000]) / len(output)

print('Accuracy:', Accuracy)

Accuracy: 0.7465753424657534
